#### Preparar Ambiente

In [1]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} python-snappy fastparquet snappy

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::seaborn-base==0.11.1=pyhd8ed1ab_1
  - conda-forge/noarch::nbclassic==0.2.6=pyhd8ed1ab_0
  - conda-forge/linux-64::blaze==0.11.3=py36_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::jupyterlab==3.0.9=pyhd8ed1ab_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/linux-64::jupyter_server==1.4.1=py36h5fab9bb_0
  - conda-forge/linux-64::bokeh==2.2.3=py36h5fab9bb_0
  - conda-forge/noarch::numpydoc==1.1.0=py_1
  - conda-forg

In [2]:
import boto3
import io
import pandas as pd

#### Seleccionar paises

In [3]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [4]:
pais = 'BR'

In [5]:
niv_adm = '4'
id_adm = 'id_adm'

#### Creando funciones

In [6]:
# Read single parquet file from S3
def pd_read_s3_parquet(key, bucket, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    return pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)

# Read multiple parquets from a folder on S3 generated by spark
def pd_read_s3_multiple_parquets(filepath, bucket, s3=None, 
                                 s3_client=None, verbose=False, **args):
    if not filepath.endswith('/'):
        filepath = filepath + '/'  # Add '/' to the end
    if s3_client is None:
        s3_client = boto3.client('s3')
    if s3 is None:
        s3 = boto3.resource('s3')
    
    s3_keys = [item.key for item in s3.Bucket(bucket).objects.filter(Prefix=filepath)]
    if not s3_keys:
        print('No parquet found in', bucket, filepath)
    elif verbose:
        print('Load parquets:')
        for p in s3_keys: 
            print(p)
    dfs = [pd_read_s3_parquet(key, bucket=bucket, s3_client=s3_client, **args) 
           for key in s3_keys]
    return pd.concat(dfs, ignore_index=True)

#### Descargando datos

In [7]:
tabla_matrizOD_adms_pais_final = f'historico_mod_adm{niv_adm}_{pais}_final'
tabla_mod = f'graphdata-mobility-public/mod/{pais}/adm{niv_adm}/{tabla_matrizOD_adms_pais_final}'
bucket = f'iadbprod-csd-hub-analyticaldata'

In [8]:
df = pd_read_s3_multiple_parquets(tabla_mod, bucket, s3_client = boto3.client('s3'))

In [9]:
df.head(10)

,year,month,day,id_adm,id_admh,n
0,2020,2,1,2932309.0,2912905.0,2
1,2020,2,1,2607703.0,2507507.0,2
2,2020,2,1,3154606.0,3143302.0,2
3,2020,2,1,3550308.0,3523602.0,2
4,2020,2,1,3550308.0,2408003.0,2
5,2020,2,1,3510500.0,2103000.0,2
6,2020,2,1,2613503.0,2613909.0,2
7,2020,2,1,3550308.0,3511409.0,2
8,2020,2,1,4120705.0,4112801.0,2
9,2020,2,1,4103701.0,4108601.0,2


In [10]:
df.to_csv('historico_mod_adm4_br.zip', compression = 'zip')

In [11]:
df.to_stata('historico_mod_adm4_br.dta')

In [12]:
df2 = pd.read_csv('historico_mod_adm4_br.zip')

In [13]:
df2

,Unnamed: 0,year,month,day,id_adm,id_admh,n
0,0,2020,2,1,2932309.0,2912905.0,2
1,1,2020,2,1,2607703.0,2507507.0,2
2,2,2020,2,1,3154606.0,3143302.0,2
3,3,2020,2,1,3550308.0,3523602.0,2
4,4,2020,2,1,3550308.0,2408003.0,2
...,...,...,...,...,...,...,...
8522141,8522141,2020,9,16,3543402.0,3519303.0,1
8522142,8522142,2020,9,16,3115102.0,3548906.0,1
8522143,8522143,2020,9,16,3304110.0,3303708.0,1
8522144,8522144,2020,9,16,3501608.0,3505500.0,1
